In [3]:
// Read the third quarter (25%) of the given dataframe.


val data = Seq(
    ("Alice", 28),
    ("Bob", 35),
    ("Charlie", 42),
    ("David", 25),
    ("Eva", 31),
    ("Frank", 38),
    ("Grace", 45),
    ("Henry", 29)
).toDF("name", "age")

data.show(false)


+-------+---+
|name   |age|
+-------+---+
|Alice  |28 |
|Bob    |35 |
|Charlie|42 |
|David  |25 |
|Eva    |31 |
|Frank  |38 |
|Grace  |45 |
|Henry  |29 |
+-------+---+



data: org.apache.spark.sql.DataFrame = [name: string, age: int]


In [28]:
import org.apache.spark.sql.expressions._

val ntiled = data.withColumn("ntiled", ntile(4).over(Window.orderBy($"age".desc))
                            ).where($"ntiled"===3).drop("ntiled")


import org.apache.spark.sql.expressions._
ntiled: org.apache.spark.sql.DataFrame = [name: string, age: int]


In [29]:
ntiled.show(false)

+-----+---+
|name |age|
+-----+---+
|Eva  |31 |
|Henry|29 |
+-----+---+



In [30]:
ntiled.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [name#34, age#35]
   +- Filter (ntiled#237 = 3)
      +- Window [ntile(4) windowspecdefinition(age#35 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS ntiled#237], [age#35 DESC NULLS LAST]
         +- Sort [age#35 DESC NULLS LAST], false, 0
            +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=457]
               +- LocalTableScan [name#34, age#35]




In [26]:
data.createOrReplaceTempView("emp")

val emp = spark.sql("""
    with cte as (
        select
            name,
            age,
            ntile(4) over(order by age desc) nt_age
        from emp
    )
    select 
        name,
        age
    from cte
    where nt_age=3
""")

emp.show(false)
emp.explain()

+-----+---+
|name |age|
+-----+---+
|Eva  |31 |
|Henry|29 |
+-----+---+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)



emp: org.apache.spark.sql.DataFrame = [name: string, age: int]
